<a href="https://colab.research.google.com/github/JaimeLopezGarrido/labo2025v/blob/main/src/ensembles/420_ArbolesAzarosos%20(Jaime).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. Ensembles de Arboles de Decision

Un arbol de decisión es un modelo débil, el aumento del poder predictivo proviene al ensamblar varios arboles de decisión.
<br> Si promedio n arboles identicos, el resultados es exactamente el mismo que utilizar un solo arbol, necesito PERTURBAR cada arbol para disponer de variablidad

la variabilidad provendrá de estas fuentes:


*   Perturbar el dataset
*   Perturbar el algoritmo del arbol
*   Perturbar el dataset y el algoritmo del arbol al mismo tiempo

Se verán estos tres algoritmos


*   Arboles Azarosos
*   Random Forest
*   Gradient Boosting of Decision Trees

#### 4.01 Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi



---



## 4.02 Arboles Azarosos

Arboles Azarosos es el nombre de un algoritmo trivial (por favor NO confundir con Random Forest)
Qué tipo de perturbaciones se realizan en Arboles Azarosos
* Se perturba el dataset
* No se perturba el algoritmo, es siempre rpart original

Cada  arbolito de  Arboles Azarosos se entrena sobre un dataset perturbado,  que tiene exactamente la misma cantidad de registros pero solo un subconjunto de los atributos (campos)  del dataset, tomados al azar, de los originales.
<br> En esta primera corrida, se construira cada arbol en un dataset utilizando el 50% de los campos

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Oct 26 12:32:03 PM 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660387,35.3,1454457,77.7,1454457,77.7
Vcells,1226637,9.4,8388608,64.0,1975128,15.1


In [3]:
# cargo las librerias que necesito
require("data.table")
require("rpart")

Loading required package: data.table

Loading required package: rpart



Aqui debe cargar SU semilla primigenia

In [5]:
PARAM <- list()
PARAM$semilla_primigenia <- 100007

# parametros  arbol
# entreno cada arbol con solo 50% de las variables variables
#  por ahora, es fijo
PARAM$feature_fraction <- 0.5

PARAM$rpart$cp <- -0.1352002
PARAM$rpart$minsplit <- 966
PARAM$rpart$minbucket <- 397
PARAM$rpart$maxdepth <- 13

# voy a generar 512 arboles,
#  a mas arboles mas tiempo de proceso y MEJOR MODELO,
#  pero ganancias marginales
PARAM$num_trees_max <- 32

In [6]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento <- "Tarea Hogar 04"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [4]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

In [8]:
# defino los dataset de entrenamiento y aplicacion
dtrain <- dataset[foto_mes == 202107]
dfuture <- dataset[foto_mes == 202109]

# arreglo clase_ternaria por algun distraido ""
dfuture[, clase_ternaria := NA ]

In [9]:
# Establezco cuales son los campos que puedo usar para la prediccion
# el copy() es por la Lazy Evaluation
campos_buenos <- copy(setdiff(colnames(dtrain), c("clase_ternaria")))

In [10]:
# que tamanos de ensemble grabo a disco
grabar <- c(1, 2, 4, 8, 16, 32, 64, 128, 256, 384, 512)

In [11]:
tb_prediccion <- dfuture[, list(numero_de_cliente)]
# aqui se va acumulando la probabilidad del ensemble
tb_prediccion[, prob_acumulada := 0]

In [12]:
set.seed(PARAM$semilla_primigenia) # Establezco la semilla aleatoria

In [13]:
for (arbolito in seq(PARAM$num_trees_max) ) {
  message( arbolito, " ")
  qty_campos_a_utilizar <- as.integer(length(campos_buenos)
    * PARAM$feature_fraction)

  # elijo los campos al azar
  campos_random <- sample(campos_buenos, qty_campos_a_utilizar)

  # paso de un vector a un string con los elementos
  # separados por un signo de "+"
  # este hace falta para la formula
  campos_random <- paste(campos_random, collapse= " + ")

  # armo la formula para rpart
  formulita <- paste0("clase_ternaria ~ ", campos_random)

  # genero el arbol de decision
  modelo <- rpart(formulita,
    data= dtrain,
    xval= 0,
    control= PARAM$rpart
  )

  # aplico el modelo a los datos que no tienen clase
  prediccion <- predict(modelo, dfuture, type= "prob")

  tb_prediccion[, prob_acumulada := prob_acumulada + prediccion[, "BAJA+2"]]

  if (arbolito %in% grabar) {
    umbral_corte <- (1 / 40) * arbolito
    tb_prediccion[, Predicted := as.numeric(prob_acumulada > umbral_corte)]

    archivo_kaggle <- paste0(
        "KA420_",
        sprintf("%.3d", arbolito), # para que tenga ceros adelante
        ".csv"
      )

    # grabo el archivo
    fwrite( tb_prediccion[, list(numero_de_cliente, Predicted)],
      file= archivo_kaggle,
      sep= ","
    )

    # subida a Kaggle
    # subida a Kaggle
    comando <- "kaggle competitions submit"
    competencia <- "-c   labo-i-2025-virtual-analista-sr"
    arch <- paste( "-f", archivo_kaggle)

    mensaje <- paste0("-m 'cp=", PARAM$rpart$cp, "  minsplit=", PARAM$rpart$minsplit, "  minbucket=", PARAM$rpart$minbucket, " maxdepth=", PARAM$rpart$maxdepth, "'" )
    linea <- paste( comando, competencia, arch, mensaje)
    salida <- system(linea, intern=TRUE)
    cat(salida)
  }
}


1 

Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c   labo-i-2025-virtual-analista-sr -f KA420_001.csv -m 'cp=-0.1352002  minsplit=966  minbucket=397 maxdepth=13'' had status 1”
2 

Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c   labo-i-2025-virtual-analista-sr -f KA420_002.csv -m 'cp=-0.1352002  minsplit=966  minbucket=397 maxdepth=13'' had status 1”
3 

4 

Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c   labo-i-2025-virtual-analista-sr -f KA420_004.csv -m 'cp=-0.1352002  minsplit=966  minbucket=397 maxdepth=13'' had status 1”
5 

6 

7 

8 

Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c   labo-i-2025-virtual-analista-sr -f KA420_008.csv -m 'cp=-0.1352002  minsplit=966  minbucket=397 maxdepth=13'' had status 1”
9 

10 

11 

12 

13 

14 

15 

16 

Warning message in system(linea, inte

In [14]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sat Oct 25 11:09:57 PM 2025"

Primer Corrida

- cp <- -1
- minsplit <- 50
- minbucket <- 20
- maxdepth <- 6

Segunda Corrida
- cp <- -0.5
- minsplit <- 20
- minbucket <- 5
- maxdepth <- 2

Tercer Corrida
- PARAM$rpart$cp <- -0.5
- PARAM$rpart$minsplit <- 400
- PARAM$rpart$minbucket <- 80
- PARAM$rpart$maxdepth <- 8

Cuarta Corrida
- PARAM$rpart$cp <- -0.1352002
- PARAM$rpart$minsplit <- 966
- PARAM$rpart$minbucket <- 397
- PARAM$rpart$maxdepth <- 13



---



# Prueba Optimizacion Bayesiana

Pruebo a ver si puedo hacer una optimizacion bayesiana para este arbol


In [5]:
install.packages("ParBayesianOptimization")
library(ParBayesianOptimization)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rbibutils’, ‘Deriv’, ‘microbenchmark’, ‘Rdpack’, ‘numDeriv’, ‘doBy’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘reformulas’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘Formula’, ‘pbkrtest’, ‘quantreg’, ‘lme4’, ‘corrplot’, ‘car’, ‘iterators’, ‘ggrepel’, ‘ggsci’, ‘cowplot’, ‘ggsignif’, ‘gridExtra’, ‘polynom’, ‘rstatix’, ‘DiceKriging’, ‘foreach’, ‘dbscan’, ‘lhs’, ‘ggpubr’




In [6]:
library(rpart)
library(data.table)

# Dataset ya cargado
dtrain <- dataset[foto_mes == 202107]
dvalid <- dataset[foto_mes == 202108]  # usamos un mes intermedio como validación

# Definimos la función objetivo
eval_arbol <- function(cp, minsplit, minbucket, maxdepth, num_trees_max) {

  PARAM <- list()
  PARAM$rpart <- list(
    cp = cp,
    minsplit = as.integer(minsplit),
    minbucket = as.integer(minbucket),
    maxdepth = as.integer(maxdepth)
  )

  PARAM$num_trees_max <- as.integer(num_trees_max)
  PARAM$feature_fraction <- 0.5

  campos_buenos <- setdiff(colnames(dtrain), "clase_ternaria")
  tb_prediccion <- dvalid[, .(numero_de_cliente)]
  tb_prediccion[, prob_acumulada := 0]

  set.seed(100007)

  for (arbolito in seq(PARAM$num_trees_max)) {
    campos_random <- sample(campos_buenos,
                            as.integer(length(campos_buenos) * PARAM$feature_fraction))
    formulita <- as.formula(paste("clase_ternaria ~", paste(campos_random, collapse = "+")))
    modelo <- rpart(formulita, data = dtrain, xval = 0, control = PARAM$rpart)
    pred <- predict(modelo, dvalid, type = "prob")
    tb_prediccion[, prob_acumulada := prob_acumulada + pred[, "BAJA+2"]]
  }

  # Usamos un umbral de corte simple
  tb_prediccion[, Predicted := as.numeric(prob_acumulada > (1 / 40) * PARAM$num_trees_max)]

  # Definí tu función de ganancia (ejemplo simple)
  ganancia <- sum(ifelse(tb_prediccion$Predicted == 1 & dvalid$clase_ternaria == "BAJA+2", 11700,
                         ifelse(tb_prediccion$Predicted == 1 & dvalid$clase_ternaria == "CONTINUA", -300, 0)))

  return(list(Score = ganancia, Pred = tb_prediccion$Predicted))
}

In [7]:
bounds <- list(
  cp = c(-0.2, 0.01),
  minsplit = c(200, 2000),
  minbucket = c(100, 1000),
  maxdepth = c(4, 20),
  num_trees_max = c(16, 64)
)

optimizacion <- bayesOpt(
  FUN = eval_arbol,
  bounds = bounds,
  initPoints = 10,    # número de puntos iniciales aleatorios
  iters.n = 30,       # cantidad de iteraciones de mejora
  acq = "ei",         # Expected Improvement
  verbose = 1
)


Running initial scoring function 10 times in 1 thread(s)...

ERROR: Error in bayesOpt(FUN = eval_arbol, bounds = bounds, initPoints = 10, : Errors encountered in initialization are listed above.


In [ ]:
getBestPars(optimizacion)
optimizacion$scoreSummary

# Segunda Prueba de Optimizacion Bayesiana

Intento codigo similar al visto en clase

In [9]:
# --- PACKAGES (instalar si hace falta) ---
# install.packages(c("data.table","rpart","mlr","mlrMBO","ParamHelpers","parallel"))
library(data.table)
library(rpart)
library(mlr)
library(mlrMBO)
library(ParamHelpers)
library(parallel)

# --- PARAMETRIZACION DEL EXPERIMENTO (como en tu ejemplo) ---
PARAM <- list()
PARAM$semilla_primigenia <- 100007
PARAM$experimento <- "Opt Bayesiana 02 Tarea 04"
PARAM$BO_iter <- 40 # iteraciones de la BO

# agrego num_trees_max y feature_fraction al espacio de búsqueda
PARAM$hs <- makeParamSet(
  makeNumericParam("cp", lower = -1, upper = 0.1),
  makeIntegerParam("minsplit", lower = 1L, upper = 8000L),
  makeIntegerParam("minbucket", lower = 1L, upper = 4000L),
  makeIntegerParam("maxdepth", lower = 3L, upper = 20L),
  makeIntegerParam("num_trees_max", lower = 8L, upper = 128L),
  makeNumericParam("feature_fraction", lower = 0.2, upper = 1.0),
  forbidden = quote(minbucket > 0.5 * minsplit)
)

# carpeta de trabajo (como en tu flujo)
setwd("/content/buckets/b1/exp")
dir.create(PARAM$experimento, showWarnings = FALSE)
setwd(paste0("/content/buckets/b1/exp/", PARAM$experimento))

# --- funciones utilitarias (copiadas y adaptadas) ---
particionar <- function(data, division, agrupa = "", campo = "fold",
                        start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)
  bloque <- unlist(mapply(function(x, y) rep(y, x), division,
                          seq(from = start, length.out = length(division))))
  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
       by = agrupa]
}

# ArbolSimple ahora entrena un ENSEMBLE (num_trees_max) sobre dtrain y evalua en dtest
ArbolSimple <- function(fold_test, param_rpart) {
  # separo train/test segun fold
  dtrain <- dataset[fold != fold_test, ]
  dtest  <- dataset[fold == fold_test, ]

  # preparo tb_prediccion para acumular probabilidades
  tb_prediccion <- dtest[, .(numero_de_cliente)]
  tb_prediccion[, prob_acumulada := 0]

# OJO con parametros establecidos mas arriba, ver si se corrieron o no. Sino se pisan los hiperparametros
  # parametros rpart (aseguro enteros donde corresponde)
  rpart_ctrl <- list(
    cp = param_rpart$cp,
    minsplit = as.integer(param_rpart$minsplit),
    minbucket = as.integer(param_rpart$minbucket),
    maxdepth = as.integer(param_rpart$maxdepth)
  )

  num_trees <- as.integer(param_rpart$num_trees_max)
  feat_frac  <- as.numeric(param_rpart$feature_fraction)

  set.seed(PARAM$semilla_primigenia + fold_test) # semilla reproducible por fold

  # campos disponibles (excluyo clase_ternaria y numero_de_cliente si existe)
  campos_buenos <- setdiff(colnames(dtrain), c("clase_ternaria", "numero_de_cliente", "fold"))

  # bucle de arboles
  for (arbolito in seq_len(num_trees)) {
    qty_campos_a_utilizar <- max(1, as.integer(length(campos_buenos) * feat_frac))
    campos_random <- sample(campos_buenos, qty_campos_a_utilizar)
    formulita <- as.formula(paste("clase_ternaria ~", paste(campos_random, collapse = " + ")))

    modelo <- rpart(formulita, data = dtrain, xval = 0, control = rpart_ctrl)

    prediccion <- predict(modelo, dtest, type = "prob")
    # si por alguna razon prediccion tiene NA, lo manejo
    if (is.null(dim(prediccion))) {
      # modelo falló; agrego 0s
      probs_baja2 <- rep(0, nrow(dtest))
    } else {
      probs_baja2 <- prediccion[, "BAJA+2"]
      probs_baja2[is.na(probs_baja2)] <- 0
    }

    tb_prediccion[, prob_acumulada := prob_acumulada + probs_baja2]
  }

  # umbral: (1/40) * num_trees   (misma lógica que tu script original)
  umbral_corte <- (1 / 40) * num_trees
  tb_prediccion[, Predicted := as.numeric(prob_acumulada > umbral_corte)]

  # calculo la ganancia en este fold (NO normalizada)
  # atención: en tu script original los premios eran 117000 y penal 3000 (aquí mantengo esa lógica)
  dtest2 <- copy(dtest)
  dtest2[, Predicted := tb_prediccion$Predicted]

  ganancia_testing <- dtest2[Predicted == 1,
                             sum(ifelse(clase_ternaria == "BAJA+2", 117000, -3000))]
  # si no hay predicciones = 1, ganancia_testing será 0 (sum de vector vacío)
  if (is.na(ganancia_testing)) ganancia_testing <- 0

  return(ganancia_testing)
}

# función que ejecuta CV con m folds y promedia (como en tu ejemplo)
ArbolesCrossValidation <- function(param_rpart, qfolds, pagrupa, semilla) {
  divi <- rep(1, qfolds)
  particionar(dataset, divi, seed = semilla, agrupa = pagrupa)
  # multiproceso: mcmapply ejecuta ArbolSimple por fold
  ganancias <- mcmapply(ArbolSimple,
                        seq(qfolds),
                        MoreArgs = list(param_rpart),
                        SIMPLIFY = FALSE,
                        mc.cores = detectCores())
  dataset[, fold := NULL]
  ganancia_promedio <- mean(unlist(ganancias))
  # normalizo igual que vos: * qfolds
  ganancia_promedio_normalizada <- ganancia_promedio * qfolds
  return(ganancia_promedio_normalizada)
}

# función objetivo para la BO: recibe una lista x con los hiperparametros
EstimarGanancia <- function(x) {
  message(format(Sys.time(), "%a %b %d %X %Y"))
  GLOBAL_iteracion <<- GLOBAL_iteracion + 1

  xval_folds <- 5
  # aseguro tipos adecuados (mlrMBO suele pasar numeric para enteros)
  x$minsplit <- as.integer(x$minsplit)
  x$minbucket <- as.integer(x$minbucket)
  x$maxdepth <- as.integer(x$maxdepth)
  x$num_trees_max <- as.integer(x$num_trees_max)
  x$feature_fraction <- as.numeric(x$feature_fraction)

  ganancia <- ArbolesCrossValidation(
    param_rpart = x,
    qfolds = xval_folds,
    pagrupa = "clase_ternaria",
    semilla = PARAM$semilla_primigenia
  )

  return(ganancia)
}

# --- LECTURA DEL DATASET (igual que tu flujo) ---
dataset <- fread("/content/datasets/dataset_pequeno.csv")

# --- LOGGING / RESUMEN para retomar corridas ---
archivo_log <- "BO_log.txt"
archivo_BO  <- "bayesian_rpart_ensemble.RDATA"

GLOBAL_iteracion <- 0
GLOBAL_mejor <- -Inf

if (file.exists(archivo_log)) {
  tabla_log <- fread(archivo_log)
  GLOBAL_iteracion <- nrow(tabla_log)
  GLOBAL_mejor <- tabla_log[, max(ganancia)]
}

# --- CONFIGURACION MLR / MBO (igual a tu ejemplo) ---
funcion_optimizar <- EstimarGanancia
configureMlr(show.learner.output = FALSE)

obj.fun <- makeSingleObjectiveFunction(
  fn = funcion_optimizar,
  minimize = FALSE,
  noisy = TRUE,
  par.set = PARAM$hs,
  has.simple.signature = FALSE
)

ctrl <- makeMBOControl(save.on.disk.at.time = 600, save.file.path = archivo_BO)
ctrl <- setMBOControlTermination(ctrl, iters = PARAM$BO_iter)
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

surr.km <- makeLearner("regr.km", predict.type = "se",
                      covtype = "matern3_2", control = list(trace = TRUE))

# ejecución (o continuación si existe archivo)
if (!file.exists(archivo_BO)) {
  bayesiana_salida <- mbo(fun = obj.fun, learner = surr.km, control = ctrl)
} else {
  bayesiana_salida <- mboContinue(archivo_BO)
}

# --- guardo resultados y extraigo mejores hiperparametros ---
tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
setorder(tb_bayesiana, -y) # ordeno por ganancia descendente

fwrite(tb_bayesiana, file = "BO_log.txt", sep = "\t")

# extraigo mejores hiperparametros (fila 1)
best_row <- tb_bayesiana[1]
PARAM$out <- list()
PARAM$out$rpart_ensemble_mejores_hiperparametros <- best_row[, .(cp, minsplit, minbucket, maxdepth, num_trees_max, feature_fraction)]

print("Mejores hiperparametros encontrados:")
print(PARAM$out$rpart_ensemble_mejores_hiperparametros)

format(Sys.time(), "%a %b %d %X %Y")

ERROR: Error in library(mlr): there is no package called ‘mlr’
